# Overview

Before building the base GRN, we need to annotate the coaccessible peaks and filter our active promoter/enhancer elements. First, we will identify the peaks around transcription starting sites (TSS). We will then merge the Cicero data with the TSS peak information and filter any peaks with weak connections to the TSS peaks. As such, the filtered peak data will only include TSS peaks and peaks with strong TSS connections. These will be our active promoter/enhancer elements for our base GRN.   
在构建基础GRN之前，我们需要对可接触区域进行注释并过滤活性的启动子/增强子元件。首先，我们将识别转录起始位点（TSS）周围的峰值。然后，我们将Cicero数据与TSS峰值信息合并，并过滤与TSS峰值连接较弱的峰值。因此，过滤后的峰值数据将仅包括TSS峰值和与TSS连接较强的峰值。这些将是我们基础GRN的活性启动子/增强子元件。

### Notebook file

Notebook file is available on CellOracle GitHub page.
https://github.com/morris-lab/CellOracle/blob/master/docs/notebooks/01_ATAC-seq_data_processing/option1_scATAC-seq_data_analysis_with_cicero/02_preprocess_peak_data.ipynb



# 0. Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns


import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm


In [2]:
from celloracle import motif_analysis as ma
import celloracle as co
co.__version__

'0.18.0'

In [3]:
%config InlineBackend.figure_format = 'retina'

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

# 1. Load scATAC peak data and peak connection data made with Cicero

## 1.1. Load data

In [5]:
# Load scATAC-seq peak list.
output_folder = "/disk1/cai026/CellOracle/HLF-Celloracle-GSM4829413/cicero_output"
peaks = pd.read_csv("/disk1/cai026/CellOracle/HLF-Celloracle-GSM4829413/cicero_output/all_peaks.csv", index_col=0)
peaks = peaks.x.values
peaks

array(['chr1_565106_565543', 'chr1_566338_566813', 'chr1_569172_569649',
       ..., 'chrY_59015854_59017543', 'chrY_59019845_59020044',
       'chrY_59030247_59030262'], dtype=object)

In [6]:
# Load Cicero coaccessibility scores.
cicero_connections = pd.read_csv("/disk1/cai026/CellOracle/HLF-Celloracle-GSM4829413/cicero_output/cicero_connections.csv", index_col=0)
cicero_connections.head()

,Peak1,Peak2,coaccess
1,chr10_100019649_100020050,chr10_99760176_99760278,0.0
2,chr10_100019649_100020050,chr10_99787982_99788292,0.0
3,chr10_100019649_100020050,chr10_99805485_99805929,0.0
4,chr10_100019649_100020050,chr10_99892674_99893222,0.0
5,chr10_100019649_100020050,chr10_99894066_99895088,0.0


# 2. Annotate transcription start sites (TSSs)¶
## IMPORTANT: Please make sure that you are setting correct reference genoms.
 If your scATAC-seq data was generated with mm10 reference genome, please set `ref_genome="mm10"`.  
 如果您的scATAC-seq数据是基于mm10参考基因组生成的，请设置`ref_genome="mm10"`.
 
You can check supported reference genome using `ma.SUPPORTED_REF_GENOME`  
你可以通过使用 `ma.SUPPORTED_REF_GENOME` 来查看支持的参考基因组。

 If your reference genome is not in the list, please send a request to us through CellOracle GitHub issue page.

In [7]:
ma.SUPPORTED_REF_GENOME

,species,ref_genome,provider
0,Human,hg38,UCSC
1,Human,hg19,UCSC
2,Mouse,mm39,UCSC
3,Mouse,mm10,UCSC
4,Mouse,mm9,UCSC
5,S.cerevisiae,sacCer2,UCSC
6,S.cerevisiae,sacCer3,UCSC
7,Zebrafish,danRer7,UCSC
8,Zebrafish,danRer10,UCSC
9,Zebrafish,danRer11,UCSC


In [8]:
##!! Please make sure to specify the correct reference genome here  注意注释的基因组
tss_annotated = ma.get_tss_info(peak_str_list=peaks, ref_genome="hg19") 

# Check results
tss_annotated.tail()

que bed peaks: 63611
tss peaks in que: 17465


,chr,start,end,gene_short_name,strand
17460,chr9,130689394,130690357,PIP5KL1,-
17461,chr9,90112515,90112827,DAPK1,+
17462,chr12,14720475,14720855,PLBD1,-
17463,chr4,156679839,156680410,GUCY1B1,+
17464,chr7,100546905,100547925,MUC3A,+


# 3. Integrate TSS info and cicero connections

The output file after the integration process has three columns: `["peak_id", "gene_short_name", "coaccess"`]. 看看输出文件里面的三列

- "peak_id" is either the TSS peak or the peaks that have a connection to a TSS peak.
- "gene_short_name" is the gene name that associated with the TSS site. 
- "coaccess" is the coaccessibility score between the peak and a TSS peak. If the score is 1, it means that the peak is a TSS itself.

In [9]:
integrated = ma.integrate_tss_peak_with_cicero(tss_peak=tss_annotated, 
                                               cicero_connections=cicero_connections)
print(integrated.shape)
integrated.head()

(188139, 3)


,peak_id,gene_short_name,coaccess
0,chr10_100028159_100028168,LOXL4,1.000000
1,chr10_100099480_100099917,PYROXD2,0.014531
2,chr10_100099480_100099917,R3HCC1L,0.011751
3,chr10_100150822_100151608,PYROXD2,0.137700
4,chr10_100150822_100151608,R3HCC1L,0.022165


# 4. Filter peaks
Remove peaks with weak coaccessibility scores.

In [10]:
peak = integrated[integrated.coaccess >= 0.8]
peak = peak[["peak_id", "gene_short_name"]].reset_index(drop=True)

In [11]:
print(peak.shape)
peak.head()

(15508, 2)


,peak_id,gene_short_name
0,chr10_100028159_100028168,LOXL4
1,chr10_100174605_100175274,PYROXD2
2,chr10_100205500_100206992,HPS1
3,chr10_100205500_100206992,LOC101927278
4,chr10_101189908_101191399,GOT1


# 5. Save data
Save the promoter/enhancer peaks.

In [12]:
peak.to_csv("/disk1/cai026/CellOracle/HLF-Celloracle-GSM4829413/cicero_output/processed_peak_file.csv")

**Please go to next step: Transcriptoin factor motif scan**

https://morris-lab.github.io/CellOracle.documentation/tutorials/motifscan.html